In [12]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
%autoreload 2

In [14]:
import ipywidgets as widgets

In [15]:
import subprocess

In [16]:
# input files widget
file_upload_widget = widgets.FileUpload(
                        accept=".txt",
                        multiple=True)

In [24]:
# parameter widget
dummy_parameter_widget = widgets.IntSlider(value=0,
                                            min=0,
                                            max=100,
                                            description="Parameter 1",
                                            )

In [18]:
# run java code widget

In [19]:
# plot results widget

In [20]:
# export results widget

In [26]:
widgets.HBox([dummy_parameter_widget, file_upload_widget])

In [33]:
display(dummy_parameter_widget.value)

0

In [39]:
file_upload_widget.value['GPLv3.txt']['content'][:dummy_parameter_widget.value]

b'                    GNU GENERAL PUBLIC LICENSE\n                       Version 3, 29 June 2007\n\n Copy'